Bureau_Balance Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import MinMaxScaler

In [2]:
Bureau_Balance = pd.read_csv('data/bureau_balance.csv')
Bureau = pd.read_csv('data/bureau.csv')
Application_Train = pd.read_csv('data/application_train.csv')
Application_Test = pd.read_csv('data/application_test.csv')

submission_id = Application_Test['SK_ID_CURR']

In [3]:
Bureau_Balance = pd.get_dummies(Bureau_Balance,columns = ['STATUS'])

In [4]:
Bureau_Balance['MONTHS_BALANCE'].unique()

array([  0,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
       -13, -14, -15, -16, -17, -18, -19, -20, -21, -22, -23, -24, -25,
       -26, -27, -28, -29, -30, -31, -32, -33, -34, -35, -36, -37, -38,
       -39, -40, -41, -42, -43, -44, -45, -46, -47, -48, -49, -50, -51,
       -52, -53, -54, -55, -56, -57, -58, -59, -60, -61, -62, -63, -64,
       -65, -66, -67, -68, -69, -70, -71, -72, -73, -74, -75, -76, -77,
       -78, -79, -80, -81, -82, -83, -84, -85, -86, -87, -88, -89, -90,
       -91, -92, -93, -94, -95, -96])

In [5]:
list_replacement_0 = [-i for i in range(12)]
list_replacement_1 = [-i for i in range(12,97)]

Bureau_Balance['MONTHS_BALANCE'] = Bureau_Balance['MONTHS_BALANCE'].replace(list_replacement_0,0)
Bureau_Balance['MONTHS_BALANCE'] = Bureau_Balance['MONTHS_BALANCE'].replace(list_replacement_1,1)

In [6]:
Bureau_Balance.isnull().sum()

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS_0          0
STATUS_1          0
STATUS_2          0
STATUS_3          0
STATUS_4          0
STATUS_5          0
STATUS_C          0
STATUS_X          0
dtype: int64

In [7]:
Bureau_Balance = Bureau_Balance.groupby('SK_ID_BUREAU', as_index = False).agg({'MONTHS_BALANCE':'mean','STATUS_0':'mean','STATUS_1':'mean','STATUS_2':'mean','STATUS_3':'mean','STATUS_4':'mean','STATUS_5':'mean','STATUS_C':'mean','STATUS_X':'mean'})

In [8]:
Bureau_Balance.head()

,SK_ID_BUREAU,STATUS_X,STATUS_5,STATUS_C,MONTHS_BALANCE,STATUS_4,STATUS_3,STATUS_2,STATUS_1,STATUS_0
0,5001709,0.113402,0.0,0.886598,0.876289,0.0,0.0,0.0,0.0,0.000000
1,5001710,0.361446,0.0,0.578313,0.855422,0.0,0.0,0.0,0.0,0.060241
2,5001711,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.750000
3,5001712,0.000000,0.0,0.473684,0.368421,0.0,0.0,0.0,0.0,0.526316
4,5001713,1.000000,0.0,0.000000,0.454545,0.0,0.0,0.0,0.0,0.000000


Grouping Bureau_Balance with the data Dataset, obtain previosuly

In [9]:
data = Bureau.merge(Bureau_Balance, on = 'SK_ID_BUREAU',how = 'left')

In [10]:
data = data.replace(['Mobile operator loan','Interbank credit','Loan for purchase of shares (margin lending)','Loan for purchase of equipment','Another type of loan','Unknown type of loan'],'Other')
data = pd.get_dummies(data,columns = ['CREDIT_ACTIVE','CREDIT_CURRENCY','CREDIT_TYPE'])
data = data.drop(['DAYS_CREDIT_UPDATE','CREDIT_CURRENCY_currency 1','CREDIT_CURRENCY_currency 2','CREDIT_CURRENCY_currency 3','CREDIT_CURRENCY_currency 4'],axis=1)
data = data.fillna(data.median())

data = data.groupby('SK_ID_CURR', as_index = False).agg({'SK_ID_BUREAU':'count','DAYS_CREDIT':'min','CREDIT_DAY_OVERDUE':'max','DAYS_CREDIT_ENDDATE':'max','DAYS_ENDDATE_FACT':'min','AMT_CREDIT_MAX_OVERDUE':['max','mean'],'CNT_CREDIT_PROLONG':['max','median'],'AMT_CREDIT_SUM':['max','median'],'AMT_CREDIT_SUM_DEBT':'sum','AMT_CREDIT_SUM_LIMIT':'max','AMT_CREDIT_SUM_OVERDUE':'sum','CREDIT_ACTIVE_Active':'count','CREDIT_ACTIVE_Bad debt':'count','CREDIT_ACTIVE_Closed':'count','CREDIT_ACTIVE_Sold':'count','CREDIT_TYPE_Car loan':'mean','CREDIT_TYPE_Cash loan (non-earmarked)':'mean','CREDIT_TYPE_Consumer credit':'mean','CREDIT_TYPE_Credit card':'mean','CREDIT_TYPE_Loan for business development':'mean','CREDIT_TYPE_Loan for the purchase of equipment':'mean','CREDIT_TYPE_Loan for working capital replenishment':'mean','CREDIT_TYPE_Microloan':'mean','CREDIT_TYPE_Mortgage':'mean','CREDIT_TYPE_Other':'mean','CREDIT_TYPE_Real estate loan':'mean','AMT_ANNUITY':'mean','MONTHS_BALANCE':'mean','STATUS_0':'mean','STATUS_1':'mean','STATUS_2':'mean','STATUS_3':'mean','STATUS_4':'mean','STATUS_5':'mean','STATUS_C':'mean','STATUS_X':'mean'})
data.columns = list(map('_'.join, data.columns.values))
data = data.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'})

In [11]:
def process(Application_Train):
    Null_dict = dict()

    for column in ['NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']:
        Null_dict[column] = 0


    for column in ['NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE']:
        if Application_Train[column].isnull().sum()>0:
            Null_dict[column] = Application_Train[column].isnull().sum()

        if (Application_Train[column]=='XNA').sum()>0:
            Null_dict[column] = (Application_Train[column]=='XNA').sum()

    for (key,value) in Null_dict.items():
        Null_dict[key] = float(value)/Application_Train.shape[0]

    def replace_binary_categorical_var(df, column_name):
        categories = list(df[column_name].unique())
        if np.nan in categories:
            categories.remove(np.nan)
        assert(len(categories) == 2)
        df.loc[df[column_name] == categories[0], column_name] = 0
        df.loc[df[column_name] == categories[1], column_name] = 1    

    for element in Application_Train['FONDKAPREMONT_MODE']:
        if element == 'XNA':
            element = np.nan
    # Name contract type is either Cash loans or Revolving loans
    replace_binary_categorical_var(Application_Train, 'NAME_CONTRACT_TYPE')

    # Gender is either male, female or N/A. We'll consider it binary
    Application_Train['CODE_GENDER'] = Application_Train['CODE_GENDER'].replace('XNA', np.nan)
    replace_binary_categorical_var(Application_Train, 'CODE_GENDER')

    # FLAG_OWN_CAR and FLAG_OWN_REALTY are flags, either Y or N
    replace_binary_categorical_var(Application_Train, 'FLAG_OWN_CAR')
    replace_binary_categorical_var(Application_Train, 'FLAG_OWN_REALTY')

    # We'll consider unknown to be N/A
    Application_Train['NAME_FAMILY_STATUS'] = Application_Train['NAME_FAMILY_STATUS'].replace('Unknown', np.nan)

    # All these are categorical
    Application_Train = pd.get_dummies(Application_Train, columns=['NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE','FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE'])

    Application_Train = Application_Train.fillna(Application_Train.median())
    return Application_Train

Application_Train = process(Application_Train)
Application_Test = process(Application_Test)

In [12]:
set(Application_Train.columns) - set(Application_Test.columns)
Application_Test['NAME_INCOME_TYPE_Maternity leave'] = 0

In [13]:
data_submission = Application_Test.merge(data, on= 'SK_ID_CURR',how='left')
data = Application_Train.merge(data, on= 'SK_ID_CURR',how='left')

In [14]:
Null_dic = dict()

for column in ['DAYS_CREDIT_min','DAYS_ENDDATE_FACT_min','AMT_CREDIT_SUM_LIMIT_max','DAYS_CREDIT_ENDDATE_max']:
    Null_dic[column] = data[column].median()


data = data.fillna(Null_dic)
data = data.fillna(0)

data_submission= data_submission.fillna(Null_dic)
data_submission= data_submission.fillna(0)

In [15]:
data_x = data.drop('TARGET', axis=1).as_matrix()
data_y = data['TARGET'].as_matrix()
data_submission = data_submission.as_matrix()

train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.05)
del data_x, data_y

In [16]:
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
data_submission = scaler.transform(data_submission)

In [17]:
xgb = XGBClassifier()
xgb.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [18]:
roc_auc_score(test_y, xgb.predict_proba(test_x)[:,1])

0.7489535742803427

In [19]:
predicted = xgb.predict_proba(data_submission)[:, 1]

predicted

results = pd.DataFrame({'SK_ID_CURR': submission_id, 'TARGET': predicted})

results.to_csv('predictions.csv', index=False)

In [20]:
data_submission

array([[-2.80699391e-06,  0.00000000e+00,  0.00000000e+00, ...,
         3.36650978e-01,  9.54545455e-04,  8.49916734e-04],
       [ 8.42098172e-06,  0.00000000e+00,  1.00000000e+00, ...,
         7.35042735e-01,  1.43636364e-03,  2.95346633e-04],
       [ 3.08769330e-05,  0.00000000e+00,  1.00000000e+00, ...,
         3.20718050e-01,  3.18750000e-03,  1.97791368e-03],
       ...,
       [ 9.99910176e-01,  0.00000000e+00,  0.00000000e+00, ...,
         3.78921569e-01,  4.87964773e-03,  2.72627661e-04],
       [ 9.99912983e-01,  0.00000000e+00,  1.00000000e+00, ...,
         2.35474147e-01,  2.50300273e-03,  7.43864574e-04],
       [ 9.99985965e-01,  0.00000000e+00,  0.00000000e+00, ...,
         1.30258630e-01,  5.43714659e-03,  2.44026748e-03]])